# Segmenting and Clustering Neighborhoods in Toronto

This notebook is divided into sections, where each section has a text explaining what the source code does.

The three questions were identified to facilitate the evaluation.

### Installation package section

This section installs all packages used throughout the notebook.

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes

print('All packages successfully installed!!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

### Library import section

This section imports all the libraries used throughout the notebook.

In [31]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing

%matplotlib inline

print('All libraries imported!')

All libraries imported!


# Part One

In [ ]:
# It is possible to use Pandas to load html data
pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [14]:
# Create dataframe by using pandas and show first 10 lines
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df[0].head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [15]:
# I've assigned index 0 to a new Pandas dataframe
df_toronto = df[0]

In [16]:
# drop cells with a borough that is Not assigned
index_NA = df_toronto[(df_toronto['Borough']=='Not assigned')].index
df_toronto.drop(index_NA, inplace=True)
df_toronto.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[df_toronto.Neighborhood=='Not assigned','Neighborhood'] = df_toronto.Neighborhood

In [18]:
df_toronto.shape

(103, 3)

### End of first part

# Part Two

In [19]:
# I used Pandas to read csv file that has the geographical coordinates of each postal code to a dataframe
# I prefered use the csv file instead of trying the Geocoder
df_coordinators = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinators.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [20]:
# I created dataframe df_ON of Toronto Neighborhood by merging two dataframes:df_toronto & df_coordinators
df_toronto = pd.merge(left=df_toronto,right=df_coordinators,on='Postal Code')
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### End of second part

# Part three

In [21]:
# Check numbers of Boroughs and Neighborhoods in the df_toronto
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape
    )
)

The dataframe has 10 boroughs and (103, 5) neighborhoods.


I had problems installing the geopy package on IBM Watson, so I fixed the latitude and longitude values.

In [22]:
# Toronto, Ontario
latitude = 43.6529
longitude = -79.3849

I've created Toronto map using Latitude and Longitude values. I've added markers to map for neighborhoods.

In [23]:
#Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

Fixing my coordinates in University of Toronto

In [28]:
# University of Toronto, Toronto, Ontario
neighborhood_latitude = 43.72149
neighborhood_longitude = -79.37881

Initialized as variables to perform a data search on Foursquare. The cells with the sensitive data were hidden.

In [35]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: <client_id>
CLIENT_SECRET:<client_secret>


In [36]:
# The code was removed by Watson Studio for sharing.

'https://api.foursquare.com/v2/venues/explore?&client_id=<client_id>&client_secret=<client_secret>&v=20180604&ll=43.72149,-79.37881&radius=500&limit=100'

In [ ]:
results = requests.get(url).json()
results

### End of third part